In [22]:
import spglib as spg
import ase
from ase.visualize import view
# the FIRE optimizer seems much better than the BFGS
from ase.optimize import BFGS, FIRE
from ase.calculators.lj import LennardJones
import ase.spacegroup as aspg
import numpy as np
import pandas as pd



import sys, os, ast
# add to PYTHONPATH

from tqdm import tqdm_notebook
# conda install -c anaconda lxml beautifulsoup4 requests

# Import Space Group info table from Seto's website with Pandas

In [2]:
url = 'http://pmsl.planet.sci.kobe-u.ac.jp/~seto/?page_id=37&lang=en'
table = pd.read_html(url)[0]

### set colomn names

In [3]:
table.columns = ['Serial No.' , 'Table No.',   'Sub No.' ,   'Main axis, Setting',  'Hall symbol' ,    'SF symbol'   ,
                  'HM symbol',   'Full notation',   'No. of Point group',  'Point group (HM)' ,   'Point group (SF)' ,  'No. of Laue Group',   
                  'Laue Group' , 'No. of Crystal System',   'Crystal System']


In [4]:
lines2del = {'hasHead':[],'isDuplicate':[]}
colName = 'Serial No.'

### Find rows where the header is duplicated

In [8]:
n,m =  table.shape
for it in range(n):
    if table.iloc[it,0] == colName:
        lines2del['hasHead'].append(it)

### Find rows of equivalent SG (there are ~550 SG and only 230 are unique)

In [9]:
sgIdx = []

for it in range(n):
    if table['Table No.'][it] not in sgIdx:
        sgIdx.append(table['Table No.'][it])
    else:
        lines2del['isDuplicate'].append(it)

### Get the SG table with relevant information

In [10]:
table = table.drop(lines2del['isDuplicate']+ lines2del['hasHead']) 

In [11]:
table.head(5)

,Serial No.,Table No.,Sub No.,"Main axis, Setting",Hall symbol,SF symbol,HM symbol,Full notation,No. of Point group,Point group (HM),Point group (SF),No. of Laue Group,Laue Group,No. of Crystal System,Crystal System
0,1,1,1,NaN,P 1,C11,P 1,P 1,1,1,C1,1,-1,1,triclinic
1,2,2,1,NaN,-P 1,Ci1,P -1,P -1,2,-1,Ci,1,-1,1,triclinic
2,3,3,1,b,P 2y,C21,P 2 = P 1 2 1,P 1 2 1,3,2,C2,2,2/m,2,monoclinic
5,6,4,1,b,P 2yb,C22,P 21 = P 1 21 1,P 1 21 1,3,2,C2,2,2/m,2,monoclinic
8,9,5,1,b1,C 2y,C23,C 2 = C 1 2 1,C 1 2 1,3,2,C2,2,2/m,2,monoclinic


In [12]:
def str2int(string):
    try:
        return int(string)
    except:
        return string

table = table.applymap(str2int)

In [13]:
ids = [it for it in range(1,230+1)]

table.reset_index()
table.index = ids
#table.head(5)

In [14]:
SgTableName = '../reference_info/SpaceGroup-general-info.pck'
table.to_pickle(SgTableName)

In [15]:
ntable = pd.read_pickle(SgTableName)
print ntable

     Serial No.  Table No.  Sub No. Main axis, Setting    Hall symbol  \
1             1          1        1                NaN            P 1   
2             2          2        1                NaN           -P 1   
3             3          3        1                  b           P 2y   
4             6          4        1                  b          P 2yb   
5             9          5        1                 b1           C 2y   
6            18          6        1                  b          P -2y   
7            21          7        1                 b1         P -2yc   
8            30          8        1                 b1          C -2y   
9            39          9        1                 b1         C -2yc   
10           57         10        1                  b          -P 2y   
11           60         11        1                  b         -P 2yb   
12           63         12        1                 b1          -C 2y   
13           72         13        1                

# Import Multiplicity and Wyckoff letter from international table for all SG

### Test on 1 SG

In [11]:
import requests
from bs4 import BeautifulSoup
#requests.get(SGurl).text
SGurl = 'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o047/'
#'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o005/'
soup = BeautifulSoup(requests.get(SGurl).text, "lxml")
content = soup.find(id="content_main")
genpos = content.findAll("table")[4]
specpos = content.findAll("table")[6]

header = ['Multiplicity','Wyckoff letter', 'Site symmetry']

In [104]:
site = []
genposList = soup.find_all("table", class_="genpos")[0]
genposcoord = genposList.find_all("td", class_="genposcoords")[0]
site.append('['+genposcoord.getText()+']')

In [105]:
specposList = soup.find_all("table", class_="specpos")[0]
trList = specposList.find_all("tr", class_="specpos")[1:]
for item in trList:
    specposcoords = item.find_all("td", class_="specposcoords")
    site.append('['+specposcoords[0].getText()+']')

In [106]:
site

[u'[(1) x,\xa0y,\xa0z]',
 u'[x,\xa0y,\xa01/2]',
 u'[x,\xa0y,\xa00]',
 u'[x,\xa01/2,\xa0z]',
 u'[x,\xa00,\xa0z]',
 u'[1/2,\xa0y,\xa0z]',
 u'[  0,\xa0y,\xa0z]',
 u'[1/2,\xa01/2,\xa0z]',
 u'[1/2,\xa00,\xa0z]',
 u'[  0,\xa01/2,\xa0z]',
 u'[  0,\xa00,\xa0z]',
 u'[1/2,\xa0y,\xa01/2]',
 u'[1/2,\xa0y,\xa00]',
 u'[  0,\xa0y,\xa01/2]',
 u'[  0,\xa0y,\xa00]',
 u'[x,\xa01/2,\xa01/2]',
 u'[x,\xa01/2,\xa00]',
 u'[x,\xa00,\xa01/2]',
 u'[x,\xa00,\xa00]',
 u'[1/2,\xa01/2,\xa01/2]',
 u'[  0,\xa01/2,\xa01/2]',
 u'[1/2,\xa01/2,\xa00]',
 u'[  0,\xa01/2,\xa00]',
 u'[1/2,\xa00,\xa01/2]',
 u'[  0,\xa00,\xa01/2]',
 u'[1/2,\xa00,\xa00]',
 u'[  0,\xa00,\xa00]']

In [107]:
def getTables(xml):
    xmlTables = xml.findAll("table")
    tableList = []
    for item in xmlTables:
        s = item.prettify()
        if '<i>' in s and '<table width="98%">' in s and 'genpos' not in s and 'specpos' not in s:
            tableList.append(item)
    return tableList

a = getTables(content)
print len(a)

27


In [109]:
a[0]

<table width="98%"><tr><td width="20%">8</td><td width="20%"><i><img alt="alpha" src="/entities/alpha_rmgif.gif"/></i></td><td width="60%">1</td></tr></table>

In [53]:
tables = content.findAll("table")
tt = []
for table in tables:
    if table.prettify().find(u'"genpos"'):
        tt.append(table)
    
print len(tt)

41


In [54]:
ttl = genpos.findAll("table")
ttt = []
for item in ttl:
    if '<i>' in item.prettify() and '<table width="98%">' in item.prettify():
        ttt.append(item)
print ttt

[]


In [55]:
a =  genpos.prettify()
print a.find(u'genpos')
print genpos

-1
<table width="98%"><tr><td width="20%">16</td><td width="20%"><i>g</i></td><td width="60%">1</td></tr></table>


In [56]:
ttt = []
ttt.extend(getTables(genpos))
ttt.extend(getTables(specpos))

In [57]:
print ttt

[<table width="98%"><tr><td width="20%">8</td><td width="20%"><i>f</i></td><td width="60%">\xa0.\xa0.\xa02\xa0</td></tr></table>, <table width="98%"><tr><td width="20%">8</td><td width="20%"><i>e</i></td><td width="60%">\xa02\xa0.\xa0.\xa0</td></tr></table>, <table width="98%"><tr><td width="20%">8</td><td width="20%"><i>d</i></td><td width="60%">\xa0<span class="overwhat"><font class="over">-</font>1</span>\xa0</td></tr></table>, <table width="98%"><tr><td width="20%">4</td><td width="20%"><i>c</i></td><td width="60%">\xa04\xa0.\xa0.\xa0</td></tr></table>, <table width="98%"><tr><td width="20%">4</td><td width="20%"><i>b</i></td><td width="60%">\xa0<span class="overwhat"><font class="over">-</font>4</span>\xa0.\xa0.\xa0</td></tr></table>, <table width="98%"><tr><td width="20%">4</td><td width="20%"><i>a</i></td><td width="60%">\xa02\xa0.\xa02\xa02\xa0</td></tr></table>]


In [131]:
from fractions import Fraction
def splitSite(string):
    return string.replace('[','').replace(']','').replace(' ','').split(',')
def evalFrac(strings):
    sss = []
    for s in strings:
        if s not in  ['x','y','z']:
            sss.append(float(Fraction(s)))
        else:
            sss.append(s)
    return sss
sss  = pd.Series(data=site).map(rmGarbageStr).map(splitSite).map(evalFrac)
print sss
#ast.literal_eval(sss[1])
print float(Fraction('1/4'))

0            [x, y, z]
1         [x, x, 0.25]
2        [0.0, 0.0, z]
3    [0.25, 0.25, 0.0]
4        [0.0, 0.5, z]
5      [0.0, 0.0, 0.0]
6     [0.0, 0.0, 0.25]
dtype: object
0.25


In [127]:
ss = splitSite(rmGarbageStr(site[1]))
sss = []
for s in ss:
    if s not in  ['x','y','z']:
        sss.append(float(Fraction(s)))
    else:
        sss.append(s)
print sss


['x', 'x', 0.25]


In [ ]:
header = ['Multiplicity','Wyckoff letter', 'Site symmetry']
table = []
for item in ttt:
    table.append(pd.read_html(str(item))[0])
print table    
table = pd.concat(table).assign(**{'Site':site})
print table 
n,m = table.shape
table.index  = range(0,n)
table.columns = header
table =  table.applymap(rmGarbageStr)
print table

In [113]:
import requests
from bs4 import BeautifulSoup
#requests.get(SGurl).text
SGurl = 'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o047/'
#'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o005/'
soup = BeautifulSoup(requests.get(SGurl).text, "lxml")
content = soup.find(id="content_main")
genposList = soup.find_all("table", class_="genpos")
specposList = soup.find_all("table", class_="specpos")

header = ['Multiplicity','Wyckoff letter', 'Site symmetry']

ttt = []
# get the data into first (the standard definition) genpos table (some space groups have several equivalent definitions)
ttt.extend(getTables(genposList[0]))
if len(specposList) > 0: # for spg 1 there are no special positions
    ttt.extend(getTables(specposList[0]))

print ttt[0]
print ttt[0].find_all("img")

<table width="98%"><tr><td width="20%">8</td><td width="20%"><i><img alt="alpha" src="/entities/alpha_rmgif.gif"/></i></td><td width="60%">1</td></tr></table>
[<img alt="alpha" src="/entities/alpha_rmgif.gif"/>]


## Working functions

In [72]:
def rmGarbageStr(string):
    """Removes '\xc2' and '\xa0' from unicode string and change the encoding to utf-8."""
    if isinstance(string,unicode):
        return string.replace(u'\xc2', '').replace(u'\xa0', '').replace(u'(1)', '').encode('utf-8')
    else:
        return string
    
def getTables(xml):
    from bs4 import BeautifulSoup
    
    xmlTables = xml.findAll("table")
    tableList = []
    
    for item in xmlTables:
        s = item.prettify()
        # these conditions seems sufficient to filter out the non relevant tables
        if '<i>' in s and '<table width="98%">' in s and 'genpos' not in s and 'specpos' not in s:
            tableList.append(item)
    return tableList

def fetchSGInfo(SpaceGroupIdx):
    import requests
    from bs4 import BeautifulSoup
    """Input the index of a space group, 
    go on the corresponding of the pages of International Tables for Crystallography Volume A: Space-group symmetry
    and returns a pandas dataframe with the Multiplicity, Wyckoff letter and Site symmetry."""
    
    # url of space group info at International Tables for Crystallography Volume A: Space-group symmetry
    baseUrl = 'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o'
    url = baseUrl + '{:03.0f}'.format(SpaceGroupIdx)+'/'

    # get the xml from the web page
    soup = BeautifulSoup(requests.get(url).text, "lxml")
    
    # get list of table of class genpos (-> general position part) and specpos (-> special positions)
    genposList = soup.find_all("table", class_="genpos")
    specposList = soup.find_all("table", class_="specpos")
    
    # get the first element of the site generator of the general position
    site = []
    genposcoord = genposList[0].find_all("td", class_="genposcoords")[0]
    site.append(genposcoord.getText())
   
    # Info that is retrieved
    header = ['Multiplicity','Wyckoff letter', 'Site symmetry','Site generator']
    
    ttt = []
    # get the data into first (the standard definition) genpos table (some space groups have several equivalent definitions)
    ttt.extend(getTables(genposList[0]))
    if len(specposList) > 0: # for spg 1 there are no special positions
        ttt.extend(getTables(specposList[0]))
        # get the first element of the site generator of the special positions
        trList = specposList[0].find_all("tr", class_="specpos")[1:]
        for item in trList:
            specposcoords = item.find_all("td", class_="specposcoords")
            site.append(specposcoords[0].getText())
        
    
    # transform the xml into a pandas dataframe and add the site information
    table = []
    for item in ttt:
        table.append(pd.read_html(str(item))[0])
    table = pd.concat(table).assign(**{'Site':site})
    
    
    
    # give a proper indexing of rows, columns headers and remove garbage from strings
    n,m = table.shape
    table.index  = range(0,n)
    table.columns = header
    table =  table.applymap(rmGarbageStr)
    
    table['Site generator'] = table['Site generator'].map(splitSite).map(evalFrac)
    
    if SpaceGroupIdx == 47:
        # This space group have 27 Wyckoff letter and on ITC the 27 position is given by alpha while on QE it is given by A
        table.loc[0,'Wyckoff letter'] = 'A'
    
    return table

def splitSite(string):
    """String manipulation for Site generator col. remove blank space and split in list of strings"""
    return string.replace(' ','').split(',')
def evalFrac(strings):
    """String manipulation for Site generator col. Interpret fractions to number, leave letters as  string."""
    from fractions import Fraction
    sss = []
    for s in strings:
        if s.find('+') > 0:
            sl = s.split('+')
            sl[1] = str(float(Fraction(sl[1])))
            sss.append(sl[0]+'+'+sl[1])
        elif s.find('-') == 0 :
            sss.append(s)
        elif s == '2x':
            sss.append('2*x')
        elif s not in  ['x','y','z']:
            sss.append(float(Fraction(s)))
        else:
            sss.append(s)
    return sss
#     return strings


## get the data

In [73]:
SGtables = {}
for sgIdx in tqdm_notebook(range(1,230+1)):
    #print sgIdx
    SGtables[sgIdx] = fetchSGInfo(sgIdx)


HBox(children=(IntProgress(value=0, max=230), HTML(value=u'')))

In [74]:
for sg in range(1,230+1):
    for it,site_gen in enumerate(SGtables[sg]['Site generator']):
        print site_gen

['x', 'y', 'z']
['x', 'y', 'z']
[0.5, 0.5, 0.5]
[0.0, 0.5, 0.5]
[0.5, 0.0, 0.5]
[0.5, 0.5, 0.0]
[0.5, 0.0, 0.0]
[0.0, 0.5, 0.0]
[0.0, 0.0, 0.5]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
[0.5, 'y', 0.5]
[0.5, 'y', 0.0]
[0.0, 'y', 0.5]
[0.0, 'y', 0.0]
['x', 'y', 'z']
['x', 'y', 'z']
[0.0, 'y', 0.5]
[0.0, 'y', 0.0]
['x', 'y', 'z']
['x', 0.5, 'z']
['x', 0.0, 'z']
['x', 'y', 'z']
['x', 'y', 'z']
['x', 0.0, 'z']
['x', 'y', 'z']
['x', 'y', 'z']
['x', 0.5, 'z']
['x', 0.0, 'z']
[0.5, 'y', 0.5]
[0.0, 'y', 0.5]
[0.5, 'y', 0.0]
[0.0, 'y', 0.0]
[0.5, 0.5, 0.5]
[0.5, 0.0, 0.5]
[0.0, 0.5, 0.5]
[0.5, 0.5, 0.0]
[0.5, 0.0, 0.0]
[0.0, 0.0, 0.5]
[0.0, 0.5, 0.0]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
['x', 0.25, 'z']
[0.5, 0.0, 0.5]
[0.0, 0.0, 0.5]
[0.5, 0.0, 0.0]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
['x', 0.0, 'z']
[0.0, 'y', 0.5]
[0.0, 'y', 0.0]
[0.25, 0.25, 0.5]
[0.25, 0.25, 0.0]
[0.0, 0.5, 0.5]
[0.0, 0.0, 0.5]
[0.0, 0.5, 0.0]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
[0.5, 'y', 0.25]
[0.0, 'y', 0.25]
[0.5, 0.0, 0.0]
[0.0, 0.5, 0.0]
[

[0.0, 0.5, 0.75]
[0.0, 0.5, 0.25]
[0.0, 0.0, 0.5]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
['x', 'x+0.5', 0.0]
[0.0, 0.5, 'z']
[0.0, 0.0, 'z']
['x', 'x', 0.25]
[0.0, 0.5, 0.0]
[0.0, 0.5, 0.25]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.25]
['x', 'y', 'z']
['x', 'x', 'z']
[0.0, 0.5, 'z']
['x', 0.0, 0.5]
['x', 0.0, 0.0]
[0.0, 0.0, 'z']
[0.0, 0.5, 0.25]
[0.0, 0.5, 0.0]
[0.0, 0.0, 0.5]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
['x', 0.25, 0.125]
[0.0, 0.0, 'z']
[0.0, 0.0, 0.5]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
['x', 0.5, 'z']
['x', 0.0, 'z']
['x', 'x', 'z']
['x', 'y', 0.5]
['x', 'y', 0.0]
['x', 0.5, 0.5]
['x', 0.5, 0.0]
['x', 0.0, 0.5]
['x', 0.0, 0.0]
['x', 'x', 0.5]
['x', 'x', 0.0]
[0.0, 0.5, 'z']
[0.5, 0.5, 'z']
[0.0, 0.0, 'z']
[0.0, 0.5, 0.0]
[0.0, 0.5, 0.5]
[0.5, 0.5, 0.5]
[0.5, 0.5, 0.0]
[0.0, 0.0, 0.5]
[0.0, 0.0, 0.0]
['x', 'y', 'z']
['x', 'y', 0.0]
['x', 0.5, 0.25]
['x', 0.0, 0.25]
['x', 'x', 0.25]
[0.0, 0.5, 'z']
[0.5, 0.5, 'z']
[0.0, 0.0, 'z']
[0.0, 0.5, 0.25]
[0.0, 0.5, 0.0]
[0.5, 0.5, 0.0]
[0.5, 0.5, 0.25]
[0.0, 

In [60]:
SGtables[191]['Site generator']

0                                 [x, y, z]
1                               [x, y, 0.5]
2                               [x, y, 0.0]
3                               [x, 2*x, z]
4                               [x, 0.0, z]
5                             [x, 2*x, 0.5]
6                             [x, 2*x, 0.0]
7                             [x, 0.0, 0.5]
8                             [x, 0.0, 0.0]
9                             [0.5, 0.0, z]
10      [0.333333333333, 0.666666666667, z]
11                          [0.5, 0.0, 0.5]
12                          [0.5, 0.0, 0.0]
13                            [0.0, 0.0, z]
14    [0.333333333333, 0.666666666667, 0.5]
15    [0.333333333333, 0.666666666667, 0.0]
16                          [0.0, 0.0, 0.5]
17                          [0.0, 0.0, 0.0]
Name: Site generator, dtype: object

In [75]:
x,y,z = [2,3,4]
for sg in range(1,230+1):
    for it,site_gen in enumerate(SGtables[sg]['Site generator']):
        for coord in site_gen:
            if isinstance(coord,float):
                continue
            else:
                try:
                    eval(coord)
                except:
                    print sg,it,site_gen 

## Save it and load it to check it is working

In [76]:
import cPickle as pck

fileName = '../reference_info/SpaceGroup-multiplicity-wickoff-info.pck'
with open(fileName, 'wb') as handle:
    pck.dump(SGtables, handle, protocol=pck.HIGHEST_PROTOCOL)

with open(fileName, 'rb') as handle:
    b = pck.load(handle)

In [77]:
b[191]

,Multiplicity,Wyckoff letter,Site symmetry,Site generator
0,24,r,1,"[x, y, z]"
1,12,q,m..,"[x, y, 0.5]"
2,12,p,m..,"[x, y, 0.0]"
3,12,o,.m.,"[x, 2*x, z]"
4,12,n,..m,"[x, 0.0, z]"
5,6,m,mm2,"[x, 2*x, 0.5]"
6,6,l,mm2,"[x, 2*x, 0.0]"
7,6,k,m2m,"[x, 0.0, 0.5]"
8,6,j,m2m,"[x, 0.0, 0.0]"
9,6,i,2mm,"[0.5, 0.0, z]"


## General info of elements

In [79]:
from mendeleev import element

In [80]:
el = element(10)

In [81]:
zatom2VdWRadiusDictD = {1: 1.2, 2: 1.43, 3: 2.12, 4: 1.98, 5: 1.91, 6: 1.77, 7: 1.66, 8: 1.5, 9: 1.46, 10: 1.58, 11: 2.5, 12: 2.51,13: 2.25, 14: 2.19, 15: 1.9, 16: 1.89, 17: 1.82, 18: 1.83, 19: 2.73, 20: 2.62, 21: 2.58, 22: 2.46, 23: 2.42,24: 2.45, 25: 2.45, 26: 2.44, 27: 2.4, 28: 2.4, 29: 2.38, 30: 2.39, 31: 2.32, 32: 2.29, 33: 1.88, 34: 1.82,35: 1.86, 36: 2.25, 37: 3.21, 38: 2.84, 39: 2.75, 40: 2.52, 41: 2.56, 42: 2.45, 43: 2.44, 44: 2.46, 45: 2.44,46: 2.15, 47: 2.53, 48: 2.49, 49: 2.43, 50: 2.42, 51: 2.47, 52: 1.99, 53: 2.04, 54: 2.06, 55: 3.48, 56: 3.03,57: 2.98, 58: 2.88, 59: 2.92, 60: 2.95, 62: 2.9, 63: 2.87, 64: 2.83, 65: 2.79, 66: 2.87, 67: 2.81, 68: 2.83,69: 2.79, 70: 2.8, 71: 2.74, 72: 2.63, 73: 2.53, 74: 2.57, 75: 2.49, 76: 2.48, 77: 2.41, 78: 2.29, 79: 2.32,80: 2.45, 81: 2.47, 82: 2.6, 83: 2.54}
electroNegativityD = {1: 2.2, 2: 0, 3: 0.98, 4: 1.57, 5: 2.04, 6: 2.55, 7: 3.04, 8: 3.44, 9: 3.98, 10: 0, 11: 0.93, 12: 1.31, 13: 1.5, 14: 1.8, 15: 2.19, 16: 2.58, 17: 3.16, 18: 0, 19: 0.82, 20: 1, 21: 1.36, 22: 1.54, 23: 1.63, 24: 1.66, 25: 1.55, 26: 1.83, 27: 1.88, 28: 1.91, 29: 1.9, 30: 1.65, 31: 1.81, 32: 2.01, 33: 2.18, 34: 2.55, 35: 2.96, 36: 0, 37: 0.82, 38: 0.95, 39: 1.22, 40: 1.33, 41: 1.6, 42: 2.16, 43: 1.9, 44: 2.2, 45: 2.28, 46: 2.2, 47: 1.93, 48: 1.69, 49: 1.78, 50: 1.96, 51: 2.05, 52: 2.1, 53: 2.66, 54: 0, 55: 0.79, 56: 0.89, 57: 1.1, 58: 1.12, 59: 1.13, 60: 1.14,  62: 1.17, 63: 1.2, 64: 1.2, 65: 1.2, 66: 1.22, 67: 1.23, 68: 1.24, 69: 1.25, 70: 1.1, 71: 1.27, 72: 1.3, 73: 1.5, 74: 2.36, 75: 1.9, 76: 2.2, 77: 2.2, 78: 2.28, 79: 2.54, 80: 2, 81: 2.04, 82: 2.33, 83: 2.02}
valenceD = {'Hydrogen':1, 'Helium':0, 'Lithium':1, 'Beryllium':2, 'Boron':3, 'Carbon':4, 'Nitrogen':3, 'Oxygen':2, 'Fluorine':1, 'Neon':0, 'Sodium':1, 'Magnesium':2, 'Aluminum':3, 'Silicon':4, 'Phosphorus':5, 'Sulfur':6, 'Chlorine':5, 'Argon':0, 'Potassium':1, 'Calcium':2, 'Scandium':3, 'Titanium':4, 'Vanadium':5, 'Chromium':6, 'Manganese':4, 'Iron':3, 'Cobalt':4, 'Nickel':2, 'Copper':2, 'Zinc':2, 'Gallium':3, 'Germanium':4, 'Arsenic':5, 'Selenium':6, 'Bromine':7, 'Krypton':2, 'Rubidium':1, 'Strontium':2, 'Yttrium':3, 'Zirconium':4, 'Niobium':5, 'Molybdenum':6, 'Technetium':7, 'Ruthenium':6, 'Rhodium':6, 'Palladium':4, 'Silver':2, 'Cadmium':2, 'Indium':3, 'Tin':4, 'Antimony':5, 'Tellurium':6, 'Iodine':7, 'Xenon':6, 'Cesium':1, 'Barium':2, 'Lanthanum':3, 'Cerium':4, 'Praseodymium':4, 'Neodymium':3, 'Samarium':3, 'Europium':3, 'Gadolinium':3, 'Terbium':3, 'Dysprosium':3, 'Holmium':3, 'Erbium':3, 'Thulium':3, 'Ytterbium':3, 'Lutetium':3, 'Hafnium':4, 'Tantalum':5, 'Tungsten':6, 'Rhenium':7, 'Osmium':6, 'Iridium':6, 'Platinum':6, 'Gold':5, 'Mercury':2, 'Thallium':3, 'Lead':4, 'Bismuth':5}
massD = {'Hydrogen':1.00794, 'Helium':4.002602, 'Lithium':6.941, 'Beryllium':9.012182, 'Boron':10.811, 'Carbon':12.0107, 'Nitrogen':14.0067, 'Oxygen':15.9994, 'Fluorine':18.9984032, 'Neon':20.1797, 'Sodium':22.98977, 'Magnesium':24.305, 'Aluminum':26.981538, 'Silicon':28.0855, 'Phosphorus':30.973761, 'Sulfur':32.065, 'Chlorine':35.453, 'Argon':39.948, 'Potassium':39.0983, 'Calcium':40.078, 'Scandium':44.95591, 'Titanium':47.867, 'Vanadium':50.9415, 'Chromium':51.9961, 'Manganese':54.938049, 'Iron':55.845, 'Cobalt':58.9332, 'Nickel':58.6934, 'Copper':63.546, 'Zinc':65.409, 'Gallium':69.723, 'Germanium':72.64, 'Arsenic':74.9216, 'Selenium':78.96, 'Bromine':79.904, 'Krypton':83.798, 'Rubidium':85.4678, 'Strontium':87.62, 'Yttrium':88.90585, 'Zirconium':91.224, 'Niobium':92.90638, 'Molybdenum':95.94, 'Technetium':98,'Ruthenium':101.07, 'Rhodium':102.9055, 'Palladium':106.42, 'Silver':107.8682, 'Cadmium':112.411, 'Indium':114.818, 'Tin':118.71, 'Antimony':121.76, 'Tellurium':127.6, 'Iodine':126.90447, 'Xenon':131.293, 'Cesium':132.90545, 'Barium':137.327, 'Lanthanum':138.9055, 'Cerium':140.116, 'Praseodymium':140.90765, 'Neodymium':144.24,'Samarium':150.36, 'Europium':151.964, 'Gadolinium':157.25, 'Terbium':158.92534, 'Dysprosium':162.5, 'Holmium':164.93032, 'Erbium':167.259, 'Thulium':168.93421, 'Ytterbium':173.04, 'Lutetium':174.967, 'Hafnium':178.49, 'Tantalum':180.9479, 'Tungsten':183.84, 'Rhenium':186.207, 'Osmium':190.23, 'Iridium':192.217, 'Platinum':195.078, 'Gold':196.96655, 'Mercury':200.59, 'Thallium':204.3833, 'Lead':207.2, 'Bismuth':208.98038}
names = ['Hydrogen', 'Helium', 'Lithium', 'Beryllium', 'Boron', 'Carbon', 'Nitrogen', 'Oxygen', 'Fluorine', 'Neon', 'Sodium', 'Magnesium', 'Aluminum', 'Silicon', 'Phosphorus', 'Sulfur', 'Chlorine', 'Argon', 'Potassium', 'Calcium', 'Scandium', 'Titanium', 'Vanadium', 'Chromium', 'Manganese', 'Iron', 'Cobalt', 'Nickel', 'Copper', 'Zinc', 'Gallium', 'Germanium', 'Arsenic', 'Selenium', 'Bromine', 'Krypton', 'Rubidium', 'Strontium', 'Yttrium', 'Zirconium', 'Niobium', 'Molybdenum', 'Technetium', 'Ruthenium', 'Rhodium', 'Palladium', 'Silver', 'Cadmium', 'Indium', 'Tin', 'Antimony', 'Tellurium', 'Iodine', 'Xenon', 'Cesium', 'Barium', 'Lanthanum', 'Cerium', 'Praseodymium', 'Neodymium', 'Samarium', 'Europium', 'Gadolinium', 'Terbium', 'Dysprosium', 'Holmium', 'Erbium', 'Thulium', 'Ytterbium', 'Lutetium', 'Hafnium', 'Tantalum', 'Tungsten', 'Rhenium', 'Osmium', 'Iridium', 'Platinum', 'Gold', 'Mercury', 'Thallium', 'Lead', 'Bismuth']

symbolD = {'Hydrogen': 'H', 'Helium': 'He', 'Lithium': 'Li', 'Beryllium': 'Be', 'Boron': 'B', 'Carbon': 'C', 'Nitrogen': 'N', 'Oxygen': 'O', 'Fluorine': 'F', 'Neon': 'Ne', 'Sodium': 'Na', 'Magnesium': 'Mg', 'Aluminum': 'Al', 'Silicon': 'Si', 'Phosphorus': 'P', 'Sulfur': 'S', 'Chlorine': 'Cl', 'Argon': 'Ar', 'Potassium': 'K', 'Calcium': 'Ca', 'Scandium': 'Sc', 'Titanium': 'Ti', 'Vanadium': 'V', 'Chromium': 'Cr', 'Manganese': 'Mn', 'Iron': 'Fe', 'Cobalt': 'Co', 'Nickel': 'Ni', 'Copper': 'Cu', 'Zinc': 'Zn', 'Gallium': 'Ga', 'Germanium': 'Ge', 'Arsenic': 'As', 'Selenium': 'Se', 'Bromine': 'Br', 'Krypton': 'Kr', 'Rubidium': 'Rb', 'Strontium': 'Sr', 'Yttrium': 'Y', 'Zirconium': 'Zr', 'Niobium': 'Nb', 'Molybdenum': 'Mo', 'Technetium': 'Tc', 'Ruthenium': 'Ru', 'Rhodium': 'Rh', 'Palladium': 'Pd', 'Silver': 'Ag', 'Cadmium': 'Cd', 'Indium': 'In', 'Tin': 'Sn', 'Antimony': 'Sb', 'Tellurium': 'Te', 'Iodine': 'I', 'Xenon': 'Xe', 'Cesium': 'Cs', 'Barium': 'Ba', 'Lanthanum': 'La', 'Cerium': 'Ce', 'Praseodymium': 'Pr', 'Neodymium': 'Nd',  'Samarium': 'Sm', 'Europium': 'Eu', 'Gadolinium': 'Gd', 'Terbium': 'Tb', 'Dysprosium': 'Dy', 'Holmium': 'Ho', 'Erbium': 'Er', 'Thulium': 'Tm', 'Ytterbium': 'Yb', 'Lutetium': 'Lu', 'Hafnium': 'Hf', 'Tantalum': 'Ta', 'Tungsten': 'W', 'Rhenium': 'Re', 'Osmium': 'Os', 'Iridium': 'Ir', 'Platinum': 'Pt', 'Gold': 'Au', 'Mercury': 'Hg', 'Thallium': 'Tl', 'Lead': 'Pb', 'Bismuth': 'Bi'}

In [82]:
# get covalent radii Pyykko, P., & Atsumi, M. (2009). Molecular Single-Bond Covalent Radii for Elements 1-118. Chemistry - A European Journal, 15(1), 186-197. doi:10.1002/chem.200800987
covalent_radius_pyykko = {}
for zatom in electroNegativityD:
    el = element(zatom)
    covalent_radius_pyykko[zatom] = el.covalent_radius_pyykko / 100.

In [83]:
print covalent_radius_pyykko

{1: 0.32, 2: 0.46, 3: 1.33, 4: 1.02, 5: 0.85, 6: 0.75, 7: 0.71, 8: 0.63, 9: 0.64, 10: 0.67, 11: 1.55, 12: 1.39, 13: 1.26, 14: 1.16, 15: 1.11, 16: 1.03, 17: 0.99, 18: 0.96, 19: 1.96, 20: 1.71, 21: 1.48, 22: 1.36, 23: 1.34, 24: 1.22, 25: 1.19, 26: 1.16, 27: 1.11, 28: 1.1, 29: 1.12, 30: 1.18, 31: 1.24, 32: 1.21, 33: 1.21, 34: 1.16, 35: 1.14, 36: 1.17, 37: 2.1, 38: 1.85, 39: 1.63, 40: 1.54, 41: 1.47, 42: 1.38, 43: 1.28, 44: 1.25, 45: 1.25, 46: 1.2, 47: 1.28, 48: 1.36, 49: 1.42, 50: 1.4, 51: 1.4, 52: 1.36, 53: 1.33, 54: 1.31, 55: 2.32, 56: 1.96, 57: 1.8, 58: 1.63, 59: 1.76, 60: 1.74, 62: 1.72, 63: 1.68, 64: 1.69, 65: 1.68, 66: 1.67, 67: 1.66, 68: 1.65, 69: 1.64, 70: 1.7, 71: 1.62, 72: 1.52, 73: 1.46, 74: 1.37, 75: 1.31, 76: 1.29, 77: 1.22, 78: 1.23, 79: 1.24, 80: 1.33, 81: 1.44, 82: 1.44, 83: 1.51}


In [84]:
zatoms = zatom2VdWRadiusDictD.keys()
VdWRadii = zatom2VdWRadiusDictD.values()
covalent_radii = covalent_radius_pyykko.values()
electroNegativities = electroNegativityD.values()
valences = [valenceD[name] for name in names]
masses = [massD[name] for name in names]
symbols = [symbolD[name] for name in names]

In [85]:
elemInfoD = {'Atomic Number':zatoms,'Symbol':symbols,'Mass [a.u.]':masses,
             'VdWRadius [angstrom]': VdWRadii,'Electronegativity': electroNegativities,'Valence': valences,'CovalentRadius [angstrom]':covalent_radii}
ee = {'z':zatoms,'sym':symbols,'mass':masses,
             'vdwr': VdWRadii,'elneg': electroNegativities,'val': valences,'covr':covalent_radii}

In [86]:
ElementsTable1 = pd.DataFrame().from_dict(elemInfoD)
ElementsTable2 = pd.DataFrame().from_dict(ee)
ElementsTable2

,covr,elneg,mass,sym,val,vdwr,z
0,0.32,2.20,1.007940,H,1,1.20,1
1,0.46,0.00,4.002602,He,0,1.43,2
2,1.33,0.98,6.941000,Li,1,2.12,3
3,1.02,1.57,9.012182,Be,2,1.98,4
4,0.85,2.04,10.811000,B,3,1.91,5
5,0.75,2.55,12.010700,C,4,1.77,6
6,0.71,3.04,14.006700,N,3,1.66,7
7,0.63,3.44,15.999400,O,2,1.50,8
8,0.64,3.98,18.998403,F,1,1.46,9
9,0.67,0.00,20.179700,Ne,0,1.58,10


In [87]:
#import cPickle as pck

fileName = '../reference_info/General-Info-Elements-clean.pck'
ElementsTable1.to_pickle(fileName)
fileName = '../reference_info/General-Info-Elements-fast.pck'
ElementsTable2.to_pickle(fileName)